In [2]:
import sympy as sp
import networkx as nx
import numpy as np
from math import factorial

import linearframework.graph_operations as g_ops
import linearframework.ca_recurrence as ca
import linearframework.linear_framework_results as lfr

## TODO

- [ ] instructions for how to quickly calculate many datapoints of a graph
- [x] tests for explicit method for k-th moment of the fpt
- [x] updated raises tests for gas.generalized_randomness_parameter(), ca.generate_sym_laplacian, g_ops.dict_to_graph()
- [ ] splitting probability
- [ ] conditional mean first passage time


In [3]:
k3_edge_to_weight = {
    ('1', '2'): 1,
    ('1', '3'): 2,
    ('2', '1'): 3,
    ('2', '3'): 4,
    ('3', '1'): 5,
    ('3', '2'): 6
}

k3 = g_ops.dict_to_graph(k3_edge_to_weight)

k3_edge_to_sym = g_ops.edge_to_sym_from_edge_to_weight(k3_edge_to_weight)

mfpt_1_3 = lfr.k_moment_fpt_expression(k3_edge_to_weight, k3_edge_to_sym, '1', '3', 1)

In [7]:
def make_sym_to_weight(edge_to_weight, edge_to_sym):
    """takes an edge-to-weight dict and an edge-to-sym dict and makes a dictionary where the symbol of an edge points to the weight pointed to by the same edge.

    Args:
        edge_to_weight (dict[tuple[str]: float]): dict of edges in form {('v_1', 'v_2): w} where w is some positive number
        edge_to_sym (dict[tuple[str]: float]): dict of edges in form {('v_1', 'v_2): l} where l is some sympy symbol

    Returns:
        dict[sp.core.symbol.Symbol: float]: symbol_to_float_dict
    """
    if not isinstance(edge_to_weight, dict):
        raise NotImplementedError("edge_to_weight must be a dictionary in the form {('v_1, 'v_2'): w} where w is a positive number and 'v_1' and 'v_2' are the ids of vertices.")
    for key in edge_to_weight.keys():
        if not isinstance(key, tuple) or not isinstance(key[0], str) or not isinstance(key[1], str) or len(key) != 2:
            raise NotImplementedError("edge_to_weight must be a dictionary in the form {('v_1, 'v_2'): w} where w is a positive number and 'v_1' and 'v_2' are the ids of vertices.")
        if not isinstance(edge_to_weight[key], (float, int)):
            raise NotImplementedError("edge_to_weight must be a dictionary in the form {('v_1, 'v_2'): w} where w is a positive number and 'v_1' and 'v_2' are the ids of vertices.")
    
    if not isinstance(edge_to_sym, dict):
        raise NotImplementedError("edge_to_sym must be a dictionary of edges to sympy symbols in the form {('v_1, 'v_2'): l_i} where l_i is a sympy symbol and 'v_1' and 'v_2' are the ids of vertices.")
    for key in edge_to_sym.keys():
        if not isinstance(key, tuple) or not isinstance(key[0], str) or not isinstance(key[1], str) or len(key) != 2:
            raise NotImplementedError("edge_to_sym must be a dictionary of edges to sympy symbols in the form {('v_1, 'v_2'): l_i} where l_i is a sympy symbol and 'v_1' and 'v_2' are the ids of vertices.")
        if not isinstance(edge_to_sym[key], sp.core.symbol.Symbol):
            raise NotImplementedError("edge_to_sym must be a dictionary of edges to sympy symbols in the form {('v_1, 'v_2'): l_i} where l_i is a sympy symbol and 'v_1' and 'v_2' are the ids of vertices.")

    sym_to_weight = {}
    for edge in edge_to_weight.keys():
        sym_to_weight[edge_to_sym[edge]] = edge_to_weight[edge]
    return sym_to_weight

symbol_to_weight = make_sym_to_weight(k3_edge_to_weight, k3_edge_to_sym)

symbol_to_weight

{l_1: 1, l_2: 2, l_3: 3, l_4: 4, l_5: 5, l_6: 6}

In [5]:
mfpt_1_3.subs(symbol_to_weight)

4/9

In [17]:
def edges_to_random_weight_dict(edges, seed=None):
    """given a list (or other iterable) of edges in the form ('v_1', 'v_2'),
    makes a dictionary with the edges as keys pointing at random weights
    sampled from the range [10**(-3), 10**6]

    Args:
        edges (iterable[tuple[str]]): iterable containing the edges

    Returns:
        dict[tuple[str]: float]: _description_
    """
    np.random.seed(seed)
    edge_to_weight = {}
    for edge in edges:
        edge_to_weight[edge] = 10 ** ((6 * np.random.rand()) - 3)
    return edge_to_weight

edges_to_random_weight_dict(k3_edge_to_weight.keys(), 1)

{('1', '2'): 0.3177840006884067,
 ('1', '3'): 20.986835607646604,
 ('2', '1'): 0.001001581395585897,
 ('2', '3'): 0.06516215458215692,
 ('3', '1'): 0.0075951323286823896,
 ('3', '2'): 0.0035812246787002297}